In [ ]:
from google.colab import files
files.upload()   # choose your kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srichakragoparaju","key":"2afb07d660b60d68cf0ca6a369134398"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sovitrath/diabetic-retinopathy-224x224-2019-data
!unzip -q diabetic-retinopathy-224x224-2019-data.zip -d dataset


Dataset URL: https://www.kaggle.com/datasets/sovitrath/diabetic-retinopathy-224x224-2019-data
License(s): CC0-1.0
 64% 153M/238M [00:00<00:00, 1.59GB/s]
100% 238M/238M [00:00<00:00, 999MB/s] 


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/content/dataset/colored_images"
train_dir = "/content/data/train"
val_dir = "/content/data/val"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for cls in os.listdir(base_dir):
    cls_path = os.path.join(base_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("✅ Split complete!")
print("Train folders:", os.listdir(train_dir))
print("Val folders:", os.listdir(val_dir))


✅ Split complete!
Train folders: ['Moderate', 'No_DR', 'Severe', 'Mild', 'Proliferate_DR']
Val folders: ['Moderate', 'No_DR', 'Severe', 'Mild', 'Proliferate_DR']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.optim import lr_scheduler
import kagglehub
import os

# Select GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------
# 1️⃣ Load the pretrained model weights
# -----------------------
print("Downloading pretrained weights...")
path = kagglehub.dataset_download("souravs17031999/blindness-detection-pretrained-weights-pytorch")
weight_path = os.path.join(path, "classifier.pt")
print("Weights path:", weight_path)

# -----------------------
# 2️⃣ Load ResNet152 and attach custom classifier
# -----------------------
model = models.resnet152(pretrained=False)
num_ftrs = model.fc.in_features
out_ftrs = 5  # 5 classes
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Linear(512, out_ftrs),
    nn.LogSoftmax(dim=1)
)

# -----------------------
# 3️⃣ Load the pretrained checkpoint (safe)
# -----------------------
checkpoint = torch.load(weight_path, map_location='cpu', weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)

model = model.to(device)
print("✅ Model loaded and ready for fine-tuning!")

# -----------------------
# 4️⃣ Define transforms and datasets
# -----------------------
train_dir = "/content/data/train"
val_dir = "/content/data/val"

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
val_data = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32)

print("✅ Datasets and loaders ready!")

# -----------------------
# 5️⃣ Define loss, optimizer, and scheduler
# -----------------------
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


Using device: cuda


100%|██████████| 617M/617M [00:05<00:00, 119MB/s]

Extracting files...


Weights path: /root/.cache/kagglehub/datasets/souravs17031999/blindness-detection-pretrained-weights-pytorch/versions/1/classifier.pt


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Model loaded and ready for fine-tuning!
✅ Datasets and loaders ready!


In [ ]:
print(train_data.class_to_idx)

{'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}


In [ ]:
import time

num_epochs = 10  # you can increase this later for better accuracy

best_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 20)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        else:
            model.eval()   # Set model to evaluation mode
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), "best_model.pth")

    scheduler.step()

print("\n✅ Training complete! Best validation accuracy: {:.4f}".format(best_acc))



Epoch 1/10
--------------------
train Loss: 1.3053 Acc: 0.6244
val Loss: 0.6762 Acc: 0.7544

Epoch 2/10
--------------------
train Loss: 0.5704 Acc: 0.8013
val Loss: 0.5451 Acc: 0.8131

Epoch 3/10
--------------------
train Loss: 0.4677 Acc: 0.8320
val Loss: 0.5045 Acc: 0.8199

Epoch 4/10
--------------------
train Loss: 0.3969 Acc: 0.8563
val Loss: 0.4912 Acc: 0.8308

Epoch 5/10
--------------------
train Loss: 0.3395 Acc: 0.8785
val Loss: 0.4613 Acc: 0.8445

Epoch 6/10
--------------------
train Loss: 0.2817 Acc: 0.8979
val Loss: 0.4650 Acc: 0.8458

Epoch 7/10
--------------------
train Loss: 0.2789 Acc: 0.8976
val Loss: 0.4595 Acc: 0.8404

Epoch 8/10
--------------------
train Loss: 0.2597 Acc: 0.9095
val Loss: 0.4591 Acc: 0.8472

Epoch 9/10
--------------------
train Loss: 0.2655 Acc: 0.8993
val Loss: 0.4682 Acc: 0.8404

Epoch 10/10
--------------------
train Loss: 0.2605 Acc: 0.9068
val Loss: 0.4595 Acc: 0.8486

✅ Training complete! Best validation accuracy: 0.8486


In [ ]:
# --- Import Required Packages ---
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

# --- Load Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Define Model Architecture ---
model = models.resnet152(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Linear(512, 5),  # 5 classes
    nn.LogSoftmax(dim=1)
)

# --- Load the Trained Weights ---
model_path = "best_model.pth"
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

print("✅ Model loaded successfully!")

# --- Define Class Labels ---
classes = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']

# --- Define Image Transformations ---
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])

# --- Inference Function ---
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    img_tensor = test_transforms(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        probabilities = torch.exp(output)
        top_p, top_class = probabilities.topk(1, dim=1)
        pred_index = top_class.item()
        confidence = top_p.item()

    print(f"\n🩺 Predicted Class: {classes[pred_index]}")
    print(f"🔢 Class Index: {pred_index}")
    print(f"💯 Confidence: {confidence:.4f}")

    return classes[pred_index], confidence

# --- Example Usage ---
# Replace this with your test image path
test_image_path = "/content/data/train/Proliferate_DR/080f66eedfb9.png"
predict_image(test_image_path)


Using device: cuda
✅ Model loaded successfully!

🩺 Predicted Class: Severe
🔢 Class Index: 3
💯 Confidence: 0.7403


('Severe', 0.7403386831283569)